# HW 2 Титаник (Дон)

In [428]:
import py4j
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.mllib.regression import LabeledPoint
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import re
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from pyspark.ml.feature import OneHotEncoder, StringIndexer

pd.set_option('display.width', 256)

### создаем спарк контест и считываем датасет

In [385]:
conf = (SparkConf().setMaster("local[8]")
        .setAppName("ML demo")
        .set("spark.executor.memory", "1g"))

In [388]:
sc = SparkContext(conf=conf)

In [389]:
sqlcontext = SQLContext(sc)

In [500]:
df = sqlcontext.read.format(
     'com.databricks.spark.csv').options(
     header='true').load('train.csv')

### отбор фич

#### 1) титул

In [501]:
def get_title(name):
    if pd.isnull(name):
        return "Null"

    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return "None"
udf_get_title = udf(get_title, StringType())
df2 = df.withColumn('title', udf_get_title(df['Name']))

stringIndexer = StringIndexer(inputCol="title", outputCol="titleIndex")
model = stringIndexer.fit(df2)
indexed = model.transform(df2)
encoder = OneHotEncoder(inputCol="titleIndex", outputCol="titleVec")
df3 = encoder.transform(indexed)

#### 2) наличие семьи(больше 3ех родственников на борту)

In [502]:
def get_family(name, parch, sibsp):
    last_name = name.split(",")[0]
    if last_name:
        family_size = 1 + int(parch) + int(sibsp)
        if family_size > 3:
            return 1
        else:
            return 0
    else:
        return 0
udf_get_family = udf(get_family, StringType())
df4 = df3.withColumn('family', udf_get_family(df3['Name'], df3['Parch'], df3['SibSp']))

#### 3) Количество родственников на борту

In [503]:
def get_relatives(name, parch, sibsp):
    last_name = name.split(",")[0]
    if last_name:
        family_size = 1 + int(parch) + int(sibsp)
        return family_size
    else:
        return 0
udf_get_relatives = udf(get_relatives, StringType())
df5 = df4.withColumn('relatives', udf_get_relatives(df3['Name'], df4['Parch'], df4['SibSp']))

#### 4) Человек-одиночка?

In [504]:
def get_single(name, parch, sibsp):
    last_name = name.split(",")[0]
    if last_name:
        family_size = 1 + int(parch) + int(sibsp)
        if family_size == 0:
            return 1
        else: 
            return 0
    else:
        return 0
udf_get_single = udf(get_single, StringType())
df6 = df5.withColumn('single', udf_get_single(df5['Name'], df5['Parch'], df5['SibSp']))

#### 5) разобьем возраст по категориям с шагом 5

In [505]:
def age_category(x):
    try:
        x = int(x)
    except:
        return -1
    return x // 5
udf_age_category = udf(age_category, IntegerType())
df7 = df6.withColumn('age_category', udf_age_category(df6['Age']))
stringIndexer = StringIndexer(inputCol="age_category", outputCol="age_categoryIndex")
model = stringIndexer.fit(df7)
indexed = model.transform(df7)
encoder = OneHotEncoder(inputCol="age_categoryIndex", outputCol="age_categoryVec")
df8 = encoder.transform(indexed)

#### заполнение пропусков поля Embarked

In [506]:
def Embarked_transform(x):
    if x != None:
        return x
    else:
        return ''

my_udf =udf(Embarked_transform, StringType())
df9 = df8.withColumn('Embarked', my_udf(df8['Embarked']))

#### создание фичей из поля Embarked - EmbarkedIndex, EmbarkedVec

In [507]:
stringIndexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
model = stringIndexer.fit(df9)
indexed = model.transform(df9)
encoder = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")
df10 = encoder.transform(indexed)

### создания нового датасета

In [508]:
def transf(r):
    return LabeledPoint(
        int(r.Survived),
        [
            int(r.Pclass),
            r.Sex == 'male',
            float(r.Fare),
            int(r.SibSp),
            int(r.Parch),
            parse_age(r.Age),
            int(r.family),
            int(r.relatives),
            int(r.family)
        ] + list(r.EmbarkedVec.toArray()) + list(r.titleVec.toArray()) + list(r.age_categoryVec.toArray())   
    )

In [509]:
data = df10.rdd.map(transf)

### обучение

In [511]:
train, test = data.randomSplit([0.7, 0.3])

In [512]:
rfc = RandomForest.trainClassifier(train, numClasses=2, categoricalFeaturesInfo={},numTrees=100)

### валидация

In [513]:
def acc(model, test):
    values = test.map(lambda x: x.features)
    yhat = model.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    errors = comp.map(lambda x: np.absolute(x[0] - x[1]))
    return 1 - errors.sum() / errors.count()

In [514]:
acc(rfc, test)

0.84782608695652173

## Конец